In [1]:
#we are goingt o be making a Movie recommendation system in python using Cosine similarity to find the closet rating . 

In [2]:
import pandas as pd    #we are using this for data handiling    
import numpy as np     #used for numerical operations
from sklearn.metrics.pairwise import cosine_similarity #we use this to find the similarity in the movies
from sklearn.feature_extraction.text import CountVectorizer #we use this to convert text data into number for algorithm processing.
from rapidfuzz import process #we use this for input matching when user makes an error  

In [3]:
#we are going to create a data frame
data_frame = pd.read_csv('IMDB-Movie-Data.csv')
#print this data framen first three rows
data_frame.head(3)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0


In [4]:
#we use the shape funcion to get the number of rows 
data_frame.shape

(1000, 12)

In [5]:
#there is no movie_id present here so we are going to manually add it 
data_frame['Movie_id'] = range(0,1000) 

In [6]:
#printing the first three rows of the data frame now with movie id
data_frame.head(3)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,Movie_id
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,1
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,2


In [7]:
#create a list of important columns for the dataset
columns = ['Title','Genre','Director','Actors']


In [8]:
data_frame[columns]

,Title,Genre,Director,Actors
0,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S..."
1,Prometheus,"Adventure,Mystery,Sci-Fi",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa..."
2,Split,"Horror,Thriller",M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar..."
3,Sing,"Animation,Comedy,Family",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma..."
4,Suicide Squad,"Action,Adventure,Fantasy",David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D..."
...,...,...,...,...
995,Secret in Their Eyes,"Crime,Drama,Mystery",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts..."
996,Hostel: Part II,Horror,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli..."
997,Step Up 2: The Streets,"Drama,Music,Romance",Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,..."
998,Search Party,"Adventure,Comedy",Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh..."


In [9]:
#check if any missing value in value in important columns
python_false = bool(data_frame[columns].isnull().values.any()) #use numpy for the values
python_false

False

In [10]:
#we concatinate data into a list 
def important_features(data):
    return data[['Title',
                'Genre',
                'Director',
                'Actors']].apply(lambda x:' '.join(x), axis=1) #apply , applies the function to the axis , aka row , and join just joins
                                                                #them in a list

In [11]:
data_frame['important_features'] = important_features(data_frame) #add the list in the data frame
#show it
data_frame.head(3)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,Movie_id,important_features
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,0,"Guardians of the Galaxy Action,Adventure,Sci-F..."
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,1,"Prometheus Adventure,Mystery,Sci-Fi Ridley Sco..."
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,2,"Split Horror,Thriller M. Night Shyamalan James..."


In [12]:
#convert the important feature list into a token count matrix
#we initalise the CountVectorizer with lowercase = True 
count_vector = CountVectorizer(lowercase=True)
transformed_count_vector = count_vector.fit_transform(data_frame['important_features'])
transformed_count_vector = transformed_count_vector.toarray()
data_frame_transformed = pd.DataFrame(data = transformed_count_vector , columns = count_vector.get_feature_names_out())
data_frame_transformed.index = data_frame['important_features']
data_frame_transformed

,10,12,127,13,13th,1408,17,2012,20th,21,...,zoë,zuk,zwart,zwick,émilie,ólafsson,ólafur,óscar,ôsugi,øvredal
important_features,,,,,,,,,,,,,,,,,,,,,
"Guardians of the Galaxy Action,Adventure,Sci-Fi James Gunn Chris Pratt, Vin Diesel, Bradley Cooper, Zoe Saldana",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Prometheus Adventure,Mystery,Sci-Fi Ridley Scott Noomi Rapace, Logan Marshall-Green, Michael Fassbender, Charlize Theron",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Split Horror,Thriller M. Night Shyamalan James McAvoy, Anya Taylor-Joy, Haley Lu Richardson, Jessica Sula",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Sing Animation,Comedy,Family Christophe Lourdelet Matthew McConaughey,Reese Witherspoon, Seth MacFarlane, Scarlett Johansson",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Suicide Squad Action,Adventure,Fantasy David Ayer Will Smith, Jared Leto, Margot Robbie, Viola Davis",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Secret in Their Eyes Crime,Drama,Mystery Billy Ray Chiwetel Ejiofor, Nicole Kidman, Julia Roberts, Dean Norris",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Hostel: Part II Horror Eli Roth Lauren German, Heather Matarazzo, Bijou Phillips, Roger Bart",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Step Up 2: The Streets Drama,Music,Romance Jon M. Chu Robert Hoffman, Briana Evigan, Cassie Ventura, Adam G. Sevani",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#naking the cos_similarity matrix from the above matrix 
cos_similarity = cosine_similarity(data_frame_transformed)
print(cos_similarity)

[[1.         0.1767767  0.06085806 ... 0.0571662  0.06537205 0.        ]
 [0.1767767  1.         0.         ... 0.         0.06933752 0.        ]
 [0.06085806 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.0571662  0.         0.         ... 1.         0.06726728 0.        ]
 [0.06537205 0.06933752 0.         ... 0.06726728 1.         0.07161149]
 [0.         0.         0.         ... 0.         0.07161149 1.        ]]


In [14]:
cos_similarity.shape

(1000, 1000)

In [16]:
data_frame['important_features'] = data_frame['Title']
def recommend_movie_title(user_input, data_frame, columns):
    movie_titles = data_frame[columns].tolist()
    closest_matches = process.extract(user_input, movie_titles, limit=5)
    print("Did you mean one of these movies?")
    for match in closest_matches:
        print(f"- {match[0]}")
    return closest_matches


def get_correct_movie_title(user_input, data_frame, columns):
    closest_matches = recommend_movie_title(user_input, data_frame, columns)
    while True:
        confirmed_movie = input("\nEnter the correct title from the list above (type 'exit' to cancel): ").strip() #we use strip to remove space before and after the word
        if confirmed_movie.lower() == 'exit':
            print("Cancelled.")
            return None
        if any(confirmed_movie.lower() == movie[0].lower() for movie in closest_matches):
            print(f"You have selected: {confirmed_movie}")
            return confirmed_movie
        else:
            print("Your choice was not recognized. Please try again.")

# Example of user input and applying the function
user_input = input("Enter the Title of the movie: ")
selected_movie = get_correct_movie_title(user_input, data_frame, 'important_features')

# Output the selected movie if not cancelled
if selected_movie:
    print(f"\nFinal selection: {selected_movie}")

#we find movie_id of your selection 
movies_id = data_frame[data_frame.Title == selected_movie]['Movie_id'].values[0]

Enter the Title of the movie:  365 days


Did you mean one of these movies?
- 3 Days to Kill
- 31
- Diary of a Wimpy Kid: Dog Days
- Cars
- The Rise of the Krays



Enter the correct title from the list above (type 'exit' to cancel):  3 Days to Kill


You have selected: 3 Days to Kill

Final selection: 3 Days to Kill


In [17]:
#create a list of enumuration of cosine similarity score
score = list(enumerate(cos_similarity[movies_id]))


In [18]:
#we sort the score so that movies with higher similarity score come first 
sorted_score = sorted(score, key=lambda x: x[1], reverse=True)
sorted_score = sorted_score[1:]

In [19]:
#we just printing the top 7 movie titles based on user re
j = 0
print(f"The seven most common movie similiar to {selected_movie} are:")
for item in sorted_score:
    Movie_title = data_frame[data_frame.Movie_id == item[0]]['Title'].values[0]
    print(j + 1, Movie_title)
    j += 1 
    if j >= 7:
        break


The seven most common movie similiar to 3 Days to Kill are:
1 Jack Ryan: Shadow Recruit
2 Criminal
3 Mr. Brooks
4 Never Back Down
5 We Need to Talk About Kevin
6 Sin City: A Dame to Kill For
7 Hanna
